In [1]:
import joblib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay

from sklearn.multiclass import OneVsRestClassifier

from collections import Counter

In [2]:
Xy_train_val = pd.read_csv('data/train.csv', index_col='review_id').fillna('Unknown')
X_train_val, y_train_val = Xy_train_val.iloc[:, :-1], Xy_train_val.iloc[:, -1].apply(lambda x: int(x[0]))

X_test = pd.read_csv('data/test.csv', index_col='review_id').fillna('Unknown')

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

X_train_val.shape, y_train_val.shape, X_test.shape

((50876, 10), (50876,), (50651, 10))

In [3]:
X_train_val.positive.apply(lambda x: len(x)).min(), X_train_val.positive.apply(lambda x: len(x)).mean(), \
X_train_val.positive.apply(lambda x: len(x)).max()

(1, 135.14995282648007, 6000)

In [4]:
Xy_train_val.target.value_counts()

8          24093
0          21003
1,8         1476
1           1269
3            905
6,8          473
6            368
7            326
3,8          209
1,6          141
5,8          121
5            102
1,5           78
1,6,8         48
4             38
4,8           36
1,5,8         33
7,8           25
1,4           24
1,7           15
5,7           13
2             12
1,3            9
5,6            9
1,5,6          7
5,6,8          6
4,6            5
3,7            5
1,4,8          4
1,4,6          3
6,7            3
1,3,8          3
1,3,5          2
1,7,8          2
1,5,6,8        1
3,6            1
5,7,8          1
4,6,8          1
1,3,6          1
3,6,8          1
3,5            1
1,2,6          1
3,5,8          1
3,5,7          1
Name: target, dtype: int64

In [5]:
y_train_val.value_counts()

8    24093
0    21003
1     3117
3     1124
6      844
7      351
5      252
4       80
2       12
Name: target, dtype: int64

In [6]:
for i in [2]: # [1, 4, 6, 7]:# [5] [2 ?] [0, 3, 8]:
    print(i)
    n = 5
    print(Xy_train_val.loc[Xy_train_val.target == str(i), 'positive'].sample(n).tolist())
    print(Xy_train_val.loc[Xy_train_val.target == str(i), 'negative'].sample(n).tolist())
    print()

2
['- Work/life balance - Возможность работать и удалённо, и в офисе (удалённая работа налажена отлично!) - Руководство - настоящие профессионалы, чуткие, открытые и идущие на встречу - Соц пакет на высшем уровне! ДМС со стоматологией, оплата мобильной связи, ноутбук, онлайн-библиотека, обучения и тренинги, система корпоративных скидок на сервисы крупных компаний практически в любой сфере, изучение АНГЛИЙСКОГО (те, кто в предыдущих отзывах писали, что его здесь нет - мы точно в одной компании работаем?)) он здесь ЕСТЬ! это и разговорные клубы, и скидка на обучение в крупнейшей онлайн платформе, и тренинги, вебинары и т.д.) - Коллектив! Тёплая атмосфера поддержки и вдохновения на всех этапах работы - Насыщенная корпоративная жизнь, праздники и мероприятия - Возможность развиваться профессионально, работать с разнообразными проектами, в том числе и с международными - Полное ощущение стабильности, всегда своевременная, белая ЗП!', 'Стабильность, доверие к компании', 'Работал мерчандайзеро

In [7]:
# how to predict 0
for i in range(9):
    pos = Xy_train_val.loc[Xy_train_val.target == str(i), 'positive']
    neg = Xy_train_val.loc[Xy_train_val.target == str(i), 'negative']
    
    print(i)
    print(pos.shape[0])
    print((pos.str.find('\xa0') == -1).mean().round(2), (neg.str.find('\xa0') == -1).mean().round(2))
#     print((pos.str.find('\xa0') == -1).astype(float).min().round(2), 
#           (neg.str.find('\xa0') == -1).astype(float).min().round(2))
#     print((pos.str.find('\xa0') == -1).astype(float).max().round(2), 
#           (neg.str.find('\xa0') == -1).astype(float).max().round(2))
    print()

0
21003
0.73 0.82

1
1269
1.0 1.0

2
12
1.0 1.0

3
905
1.0 1.0

4
38
1.0 1.0

5
102
1.0 1.0

6
368
1.0 1.0

7
326
1.0 1.0

8
24093
1.0 1.0



In [8]:
# how to predict 2 ???
for i in range(9):
    pos = Xy_train_val.loc[Xy_train_val.target == str(i), 'positive']
    neg = Xy_train_val.loc[Xy_train_val.target == str(i), 'negative']
    
    print(i)
    print(pos.shape[0])
    print(pos.apply(lambda x: Counter([w for w in x.split(' ')]).most_common(1)[0][1]).median(), 
          neg.apply(lambda x: Counter([w for w in x.split(' ')]).most_common(1)[0][1]).median())
#     print(pos.apply(lambda x: Counter([len(w) for w in x.split(' ')]).most_common(1)[0][1]).min(), 
#           neg.apply(lambda x: Counter([len(w) for w in x.split(' ')]).most_common(1)[0][1]).min())
#     print(pos.apply(lambda x: Counter([len(w) for w in x.split(' ')]).most_common(1)[0][1]).max(), 
#           neg.apply(lambda x: Counter([len(w) for w in x.split(' ')]).most_common(1)[0][1]).max())
    print()

0
21003
1.0 1.0

1
1269
1.0 1.0

2
12
3.0 2.0

3
905
1.0 1.0

4
38
1.0 2.0

5
102
1.0 1.0

6
368
1.0 2.0

7
326
1.0 1.0

8
24093
1.0 1.0



In [9]:
# how to predict 3
for i in range(9):
    pos = Xy_train_val.loc[Xy_train_val.target == str(i), 'positive']
    neg = Xy_train_val.loc[Xy_train_val.target == str(i), 'negative']
    
    print(i)
    print(pos.shape[0])
    print(pos.apply(lambda x: np.max([len(w) for w in x.split(' ')])).median(), 
          neg.apply(lambda x: np.max([len(w) for w in x.split(' ')])).median())
#     print(pos.apply(lambda x: np.max([len(w) for w in x.split(' ')])).min(), 
#           neg.apply(lambda x: np.max([len(w) for w in x.split(' ')])).min())
#     print(pos.apply(lambda x: np.max([len(w) for w in x.split(' ')])).max(), 
#           neg.apply(lambda x: np.max([len(w) for w in x.split(' ')])).max())
    print()

0
21003
13.0 11.0

1
1269
12.0 12.0

2
12
12.5 11.5

3
905
21.0 21.0

4
38
13.0 13.0

5
102
12.0 12.0

6
368
12.0 13.0

7
326
12.0 12.0

8
24093
11.0 11.0



In [10]:
# how to predict 5
for i in range(9):
    pos = Xy_train_val.loc[Xy_train_val.target == str(i), 'positive']
    neg = Xy_train_val.loc[Xy_train_val.target == str(i), 'negative']
    
    print(i)
    print(pos.shape[0])
    print(pos.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).median(), 
          neg.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).median())
#     print(pos.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).min(), 
#           neg.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).min())
#     print(pos.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).max(), 
#           neg.apply(lambda x: np.max([sum(1 for c in s if c.isupper()) for s in x.split('.')])).max())
    print()

0
21003
1.0 1.0

1
1269
1.0 1.0

2
12
1.0 1.0

3
905
0.0 1.0

4
38
1.0 1.0

5
102
2.0 2.0

6
368
1.0 1.0

7
326
1.0 1.0

8
24093
1.0 1.0



In [11]:
# how to predict 8
for i in range(9):
    pos = Xy_train_val.loc[Xy_train_val.target == str(i), 'positive']
    neg = Xy_train_val.loc[Xy_train_val.target == str(i), 'negative']
    
    print(i)
    print(pos.shape[0])
    print(pos.apply(lambda x: len(x)).mean(), 
          neg.apply(lambda x: len(x)).mean())
    print(pos.apply(lambda x: len(x)).median(), 
          neg.apply(lambda x: len(x)).median())
    print(pos.apply(lambda x: len(x)).min(), 
          neg.apply(lambda x: len(x)).min())
    print(pos.apply(lambda x: len(x)).max(), 
          neg.apply(lambda x: len(x)).max())
    print()

0
21003
245.46436223396657 89.81964481264582
191.0 52.0
14 1
4413 2668

1
1269
115.08431836091411 193.60520094562648
66.0 95.0
20 20
2882 2854

2
12
351.5 113.08333333333333
331.0 126.0
22 31
935 176

3
905
64.7414364640884 61.90276243093923
26.0 27.0
7 7
6000 6000

4
38
130.81578947368422 276.1842105263158
82.5 163.5
22 28
793 2735

5
102
189.34313725490196 241.4607843137255
112.0 100.5
20 21
1002 3706

6
368
129.22554347826087 309.625
76.0 194.5
20 21
1145 3591

7
326
178.0245398773006 167.24846625766872
82.5 72.0
20 20
2369 1561

8
24093
45.4064666085585 61.878553936828126
34.0 39.0
1 1
3551 4461



In [12]:
for i in range(9):
    print(i)
    n = 8
    print(Xy_train_val.loc[Xy_train_val.target == str(i), 'position'].sample(n).tolist())
    print(Xy_train_val.loc[Xy_train_val.target == str(i), 'city'].sample(n).tolist())
    print()

0
['Менеджер', 'Менеджер офиса продаж', 'Кредитный инспектор отдела промышленности и энергетики', 'Demand Planning Specialist', 'Инженер', 'Unknown', 'Project Manager', 'Unknown']
['Unknown', 'Казань', 'Тюмень', 'Санкт-Петербург', 'Москва', 'Москва', 'Петрозаводск', 'Москва']

1
['Специалист', 'Помощник машиниста', 'Повар пиццы', 'Продавец-консультант', 'Unknown', 'Старший инженер', 'Кассир билетный на ЖД транспорте', 'Кассир']
['Москва', 'Саратов', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва', 'Нижний Новгород', 'Москва']

2
['Unknown', 'Провизор', 'Специалист по страхованию', 'Unknown', 'Рекрутер', 'Территориальный управляющий', 'Мерчандайзер', 'Консультант-эксперт']
['Ростов-на-Дону', 'Москва', 'Unknown', 'Санкт-Петербург', 'Калининград', 'Пермь', 'Москва', 'Unknown']

3
['Оператор ПК', 'Эксперт клиентского центра', 'Продавец-консультант', 'Техник по наладке и ремонту бурового оборудования 2 категорий', 'Продавец-консультант', 'Помощник руководителя ломбардно-скупочной деятельнос

In [13]:
Xy_train_val.columns

Index(['city', 'position', 'positive', 'negative', 'salary_rating',
       'team_rating', 'managment_rating', 'career_rating', 'workplace_rating',
       'rest_recovery_rating', 'target'],
      dtype='object')

In [14]:
for i in range(9):
#     print(i, col)
    n = 8
    print(Xy_train_val.loc[Xy_train_val.target == str(i), 
                           ['salary_rating', 'team_rating', 'managment_rating', 'career_rating', 
                            'workplace_rating', 'rest_recovery_rating']
                          ].rename(columns = {'salary_rating': 'salary', 
                                              'team_rating': 'team', 
                                              'managment_rating': 'managment', 
                                              'career_rating':'career', 
                                              'workplace_rating': 'workplace', 
                                              'rest_recovery_rating': 'rest_recovery'}).sample(n))
    print()

           salary  team  managment  career  workplace  rest_recovery
review_id                                                           
14597           5     5          5       5          5              5
4609            5     5          5       5          5              5
23991           4     4          3       4          4              4
158303          5     5          2       5          5              4
61180           5     5          5       5          5              5
110011          3     5          4       5          5              5
115097          5     4          4       5          4              4
78215           5     5          5       5          5              5

           salary  team  managment  career  workplace  rest_recovery
review_id                                                           
122782          2     4          3       1          4              3
7386            1     1          1       1          1              1
54947           1     4          